In [1]:
from rdkit import Chem
from rdkit.Chem import AllChem
import py3Dmol
from rdkit.Chem import Draw
from rdkit.Chem.Draw import IPythonConsole

import prody as pr

IPythonConsole.ipython_useSVG=True 
IPythonConsole.drawOptions.addAtomIndices = True



In [ ]:
workdir = '/Users/lonelu/DesignData/fragment_design/FragmentScreen_yuda/'

#ligand = pr.parsePDB(workdir + '7OCoumarin fragment ABLE.pdb').select('resname LIG')

#m_start = Chem.MolFromPDBFile(workdir + 'm_start_conf.pdb')
#m_start = Chem.MolFromPDBFile(workdir + 'CMR.pdb')
suppl = Chem.SDMolSupplier(workdir + 'CMR.sdf')
m_start = [x for x in suppl][0]
#m_start = Chem.MolFromSmiles('Oc1ccc2ccc(=O)oc2c1')

molpath = workdir +  'molport_2.sdf'

suppl = Chem.SDMolSupplier(molpath)

mols = [x for x in suppl]

print(len(mols))

In [ ]:
for m in mols[:8]: tmp=AllChem.Compute2DCoords(m)
img=Draw.MolsToGridImage(mols[:6],molsPerRow=2,subImgSize=(200,200),legends=[x.GetProp("_Name") for x in mols[:6]])  

img

In [ ]:

hasSubs = [m.HasSubstructMatch(m_start) for m in mols]

print(sum(hasSubs))

print(hasSubs[:10])

In [ ]:
m = mols[0]
match_atom_ids = m.GetSubstructMatches(m_start)

print(match_atom_ids)

m2 = mols[2]

atomMap = {}

match_atom_ids2 = m2.GetSubstructMatches(m_start)
print(match_atom_ids2)

m

In [ ]:
# def mol_with_atom_index(mol):
#     for atom in mol.GetAtoms():
#         atom.SetAtomMapNum(atom.GetIdx())
#     return mol

#mol_with_atom_index(m_start)

In [ ]:
#print(Chem.MolToMolBlock(m_start))  
conf = m_start.GetConformer()
print(conf.GetPositions())  
IPythonConsole.drawMol3D(m_start)

In [ ]:
# murckoHashList = [Chem.rdMolHash.MolHash(mMol, Chem.rdMolHash.HashFunction.MurckoScaffold) for mMol in mols[:6]]
# highlight_mostFreq_murckoHash = [m.GetSubstructMatch(m_start) for m in mols[:6]]
# Draw.MolsToGridImage(mols,legends=[murckoHash for murckoHash in murckoHashList],
#                      highlightAtomLists = highlight_mostFreq_murckoHash,
#                      subImgSize=(250,250), useSVG=False)


In [ ]:
# generate confomers.
m = Chem.AddHs(m)
cids = AllChem.EmbedMultipleConfs(m, numConfs=64, randomSeed=0xf00d)
print(list(cids))
#rmslist = []
#AllChem.AlignMolConformers(m, RMSlist=rmslist)



In [ ]:
heavy_atom_num = Chem.rdMolDescriptors.CalcNumHeavyAtoms(m)
print(heavy_atom_num)

m.GetAtoms()[41].GetSymbol()
m.GetAtoms()[42].GetSymbol()

In [ ]:
import numpy as np
def get_mol_coords(mol_mobile, cids):
    coords = []
    heavy_atom_num = Chem.rdMolDescriptors.CalcNumHeavyAtoms(mol_mobile)
    for id in cids:
        conf = mol_mobile.GetConformer(id)
        coord = conf.GetPositions()[:heavy_atom_num]
        coords.append(coord)
    return np.array(coords).reshape(-1, 3)


coords =  get_mol_coords(m, cids)
print(coords.shape)

In [ ]:
conf1_3 = m.GetConformer(3)
print(conf1_3.GetPositions()[35:38]) 
conf1_4 = m.GetConformer(4)
print(conf1_4.GetPositions()[35:38])   


In [ ]:
m

In [ ]:
_coordMap = {}
print(m_start_conf.GetAtomPosition(0))

m_start_conf = m_start.GetConformer()
for i in range(len(match_atom_ids2[0])):
    x = match_atom_ids2[0][i]
    #atomMap[x] = m_start_conf.GetId() 
    _coordMap[x] = m_start_conf.GetAtomPosition(i) 

print(_coordMap)

In [ ]:
m2= Chem.AddHs(m2)
cids2 = AllChem.EmbedMultipleConfs(m2, numConfs=64, coordMap = _coordMap)
print(cids2[1])

In [ ]:
conf2_3 = m2.GetConformer(3)
print(conf2_3.GetPositions()[17:20]) 

conf2_4 = m2.GetConformer(4)
print(conf2_4.GetPositions()[17:20])  

In [ ]:
_atomMap = [(match_atom_ids[0][i], i) for i in range(len(match_atom_ids[0]))]
print(_atomMap)

Chem.rdMolAlign.AlignMol(m, m_start, prbCid = 3, atomMap = _atomMap)


In [ ]:
print(conf.GetPositions())  

In [ ]:
from rdkit.Chem.rdmolfiles import MolToPDBFile

MolToPDBFile(m_start, workdir + 'm_start_conf' + '.pdb', confId = -1)

MolToPDBFile(m, workdir + 'm1-3_conf.pdb', confId = 3)
MolToPDBFile(m, workdir + 'm1-4_conf.pdb', confId = 4)

MolToPDBFile(m2, workdir + 'm2-3_conf.pdb', confId = 3)
MolToPDBFile(m2, workdir + 'm2-4_conf.pdb', confId = 4)

In [ ]:
print(m.GetProp('_Name'))

xs = mols[0].GetPropNames()
for x in xs:
    print(x)

mols[0].GetProp('Molport ID')

In [2]:
workdir = '/Users/lonelu/DesignData/fragment_design/FragmentScreen_yuda/'

molpath = workdir +  'Coumarin_Molport_10000.sdf'

suppl = Chem.SDMolSupplier(molpath)

with open(workdir + 'Coumarin_Molport_10000.txt', 'w') as f:
    for m in suppl:
        f.write(m.GetProp('Molport ID') + '\n')

